In [64]:
import os
import pandas as pd
from supabase import create_client, Client

url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [65]:
import geopandas as gpd
from shapely.geometry import Point



response = supabase.table('entrances').select("*").execute()
data,_ = response

entrances_df = pd.DataFrame(data[1])
# Create a new column in your DataFrame for the geographic data
entrances_df['geometry'] = [Point(xy) for xy in zip(entrances_df['longitude'], entrances_df['latitude'])]

# Convert the DataFrame to a GeoDataFrame
entrances_gdf = gpd.GeoDataFrame(entrances_df, geometry='geometry')
# Set the coordinate reference system (CRS) to EPSG:4326 (WGS84)
entrances_gdf.crs = "EPSG:4326"

entrances_gdf

,entrance_id,longitude,latitude,entrance_destination,entrance_name,geometry
0,3308608988,101.712717,3.158762,None,None,POINT (101.71272 3.15876)
1,3308608989,101.712507,3.158809,None,None,POINT (101.71251 3.15881)
2,4092013971,101.614130,3.022231,None,None,POINT (101.61413 3.02223)
3,4952299503,101.732720,3.104671,None,B,POINT (101.73272 3.10467)
4,5040821684,101.711729,3.146919,None,D,POINT (101.71173 3.14692)
...,...,...,...,...,...,...
284,12076281187,101.681477,3.113727,None,None,POINT (101.68148 3.11373)
285,12076314945,101.704860,3.098311,None,None,POINT (101.70486 3.09831)
286,12084946657,101.710926,3.076792,None,None,POINT (101.71093 3.07679)
287,12084946658,101.711568,3.075897,None,None,POINT (101.71157 3.07590)


In [66]:
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import os

import sys

# Get the path of the parent directory (the root of the project)
module_path = os.path.abspath(os.path.join('..'))

# Add the parent directory to sys.path
sys.path.append(module_path)

# Now you can import key.py
from key import ORS_KEY

#Personal api_key stored in a config.py file that is in gitignore. Uncomment the following and provide your own ORS api for your own use
## api_key = "you api key " #Provide your personal API key
ors_key: str = ORS_KEY


ors_client = client.Client(key=ors_key) 

In [67]:
[entrances_gdf.iloc[0]['geometry'].coords[0][0],entrances_gdf.iloc[0]['geometry'].coords[0][1]]

[101.7127175, 3.1587619]

In [71]:
isochrone_params = {
              'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
isochrone_params['locations'] = [[entrances_gdf.iloc[0]['geometry'].coords[0][0],entrances_gdf.iloc[0]['geometry'].coords[0][1]]]

temp_iso = ors_client.isochrones(**isochrone_params)


In [72]:
import json
data = json.loads(json.dumps(temp_iso))
df = pd.json_normalize(data['features'])
df = df.drop(columns=['type','properties.group_index'])
df

,properties.value,properties.center,properties.area,properties.reachfactor,properties.total_pop,geometry.coordinates,geometry.type
0,300.0,"[101.71272677986722, 3.158791494667739]",356944.49,1.0000,5009.0,"[[[101.709289, 3.158813], [101.709337, 3.15874...",Polygon
1,600.0,"[101.71272677986722, 3.158791494667739]",1365002.74,0.9776,19193.0,"[[[101.705986, 3.157366], [101.70598, 3.157247...",Polygon
2,900.0,"[101.71272677986722, 3.158791494667739]",2962453.45,0.9430,42510.0,"[[[101.702889, 3.158236], [101.702432, 3.15634...",Polygon


In [73]:
df['geometry.coordinates']=df['geometry.coordinates'].apply(lambda row:row[0])

# Remove 'properties.' and 'geometry.' from the column names
df.columns = df.columns.str.replace('properties.', '', regex=False)
df.columns = df.columns.str.replace('geometry.', '', regex=False)
df

,value,center,area,reachfactor,total_pop,coordinates,type
0,300.0,"[101.71272677986722, 3.158791494667739]",356944.49,1.0000,5009.0,"[[101.709289, 3.158813], [101.709337, 3.15874]...",Polygon
1,600.0,"[101.71272677986722, 3.158791494667739]",1365002.74,0.9776,19193.0,"[[101.705986, 3.157366], [101.70598, 3.157247]...",Polygon
2,900.0,"[101.71272677986722, 3.158791494667739]",2962453.45,0.9430,42510.0,"[[101.702889, 3.158236], [101.702432, 3.156343...",Polygon


In [74]:

df['entrance_id']=entrances_gdf.iloc[0]['entrance_id']
df['isochrone_id'] = df['entrance_id'].astype(str) + df['value'].astype(int).astype(str)
df

,value,center,area,reachfactor,total_pop,coordinates,type,entrance_id,isochrone_id
0,300.0,"[101.71272677986722, 3.158791494667739]",356944.49,1.0000,5009.0,"[[101.709289, 3.158813], [101.709337, 3.15874]...",Polygon,3308608988,3308608988300
1,600.0,"[101.71272677986722, 3.158791494667739]",1365002.74,0.9776,19193.0,"[[101.705986, 3.157366], [101.70598, 3.157247]...",Polygon,3308608988,3308608988600
2,900.0,"[101.71272677986722, 3.158791494667739]",2962453.45,0.9430,42510.0,"[[101.702889, 3.158236], [101.702432, 3.156343...",Polygon,3308608988,3308608988900


In [75]:
from shapely.geometry import Polygon

# Convert list of coordinates to Polygon objects
df['geometry'] = df['coordinates'].apply(Polygon)

# Convert to GeoDataFrame
entrances_iso_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")



In [76]:
entrances_iso_gdf

,value,center,area,reachfactor,total_pop,coordinates,type,entrance_id,isochrone_id,geometry
0,300.0,"[101.71272677986722, 3.158791494667739]",356944.49,1.0000,5009.0,"[[101.709289, 3.158813], [101.709337, 3.15874]...",Polygon,3308608988,3308608988300,"POLYGON ((101.70929 3.15881, 101.70934 3.15874..."
1,600.0,"[101.71272677986722, 3.158791494667739]",1365002.74,0.9776,19193.0,"[[101.705986, 3.157366], [101.70598, 3.157247]...",Polygon,3308608988,3308608988600,"POLYGON ((101.70599 3.15737, 101.70598 3.15725..."
2,900.0,"[101.71272677986722, 3.158791494667739]",2962453.45,0.9430,42510.0,"[[101.702889, 3.158236], [101.702432, 3.156343...",Polygon,3308608988,3308608988900,"POLYGON ((101.70289 3.15824, 101.70243 3.15634..."


In [ ]:
def isochrone_generate(client,parameters,reach_centre):
    import json

    #reach_centre is in the format: [101.7127175, 3.1587619], ie [longitude,latitude]
    parameters['locations'] = reach_centre 
    isochrones_output = client.isochrones(**parameters)

    #convert query output to json
    data_json = json.loads(json.dumps(isochrones_output))

    #json to dataframe
    df = pd.json_normalize(data_json['features'])
    df = df.drop(columns=['type','properties.group_index'])

    #reformate coordinates column
    df['geometry.coordinates']=df['geometry.coordinates'].apply(lambda row:row[0])

    # Remove 'properties.' and 'geometry.' from the column names
    df.columns = df.columns.str.replace('properties.', '', regex=False)
    df.columns = df.columns.str.replace('geometry.', '', regex=False)
    
    return df    